# NeMo SD - oracleVAD ami mixheadset 15.7%

In [1]:
# """
# You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.
# TitaNet: https://arxiv.org/pdf/2110.04410.pdf

# Instructions for setting up Colab are as follows:
# https://colab.research.google.com/drive/1ZoALe4Vy4ekR3eedVwyyt1zF8ZjPZ2JU#scrollTo=sDtW2e8u634q
# 1. Open a new Python 3 notebook.
# 2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
# 3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
# 4. Run this cell to set up dependencies.
# """
# # If you're using Google Colab and not running locally, run this cell.

# ## Install dependencies
# !pip install wget
# !apt-get install sox libsndfile1 ffmpeg
# !pip install unidecode

# # ## Install NeMo
# BRANCH = 'main'
# !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]

# ## Install TorchAudio
# !pip install torchaudio -f https://download.pytorch.org/whl/torch_stable.html
# # !pip install torch==1.11.0 torchaudio==0.11.0 torchvision==0.12.0

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# CUDA_VISIBLE_DEVICES options = 0/1/2/3. Make sure to restart
!echo $CUDA_VISIBLE_DEVICES

3


In [2]:
import os
import glob
import subprocess
import tarfile
import wget
import pandas as pd
from collections import Counter
from omegaconf import OmegaConf
import numpy as np
import spyder

import IPython
from pyannote.metrics.diarization import DiarizationErrorRate
from nemo.collections.asr.parts.utils.speaker_utils import rttm_to_labels, labels_to_pyannote_object
from nemo.collections.asr.models import ClusteringDiarizer


/root/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
[NeMo W 2022-07-22 13:24:15 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.


In [3]:
ROOT = os.getcwd()
data_dir = os.path.join(ROOT,'data/ami_mix_headset')
os.makedirs(data_dir, exist_ok=True)
print("ROOT: ", ROOT)
print("Data Directory: ", data_dir)
      
audio_filepath = '/home/DATA/amit_kesari/AMI-diarization-setup/pyannote/amicorpus/'
groundtruth_rttm_filepath = os.path.join(data_dir, 'rttms')
print(groundtruth_rttm_filepath)
# has a train,dev,test subdir

if not os.path.exists(audio_filepath):
    print("Please give correct audio location. No ", audio_filepath)
if not os.path.exists(groundtruth_rttm_filepath):
    print("Please give correct rttm groundtruth location. No ", groundtruth_rttm_filepath)

    
test_type = 'test'
groundtruth_rttm_list = []
if test_type == 'test':
    groundtruth_rttm_list = glob.glob(groundtruth_rttm_filepath + '/test/*.rttm', recursive=True)
    for test_path in groundtruth_rttm_list:
        print(test_path)

!find {audio_filepath} -iname "*.wav" > data/ami_mix_headset/train_all.txt
# !cat {data_dir}/ami_headset/train_all.txt
# DEMO_FILE = {'uri': 'ES2004a.Mix-Headset', 'audio': 'ES2004a.Mix-Headset.wav'}


audio_list = []
with open(data_dir+'/train_all.txt') as data:
    audio_list.append(data.read().split())
audio_list = audio_list[0]
audio_list

ROOT:  /home/DATA/amit_kesari/SD1/NeMo-Nvidia
Data Directory:  /home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset
/home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/rttms
/home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/rttms/test/TS3003c.Mix-Headset.rttm
/home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/rttms/test/EN2002a.Mix-Headset.rttm
/home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/rttms/test/EN2002b.Mix-Headset.rttm
/home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/rttms/test/EN2002c.Mix-Headset.rttm
/home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/rttms/test/EN2002d.Mix-Headset.rttm
/home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/rttms/test/ES2004a.Mix-Headset.rttm
/home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/rttms/test/ES2004b.Mix-Headset.rttm
/home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/rttms/test/ES2004c.Mix-Headset.rttm
/home/DATA/amit_kesari/SD1/NeMo-Nvidia/data

['/home/DATA/amit_kesari/AMI-diarization-setup/pyannote/amicorpus/ES2002a/audio/ES2002a.Mix-Headset.wav',
 '/home/DATA/amit_kesari/AMI-diarization-setup/pyannote/amicorpus/ES2002b/audio/ES2002b.Mix-Headset.wav',
 '/home/DATA/amit_kesari/AMI-diarization-setup/pyannote/amicorpus/ES2002c/audio/ES2002c.Mix-Headset.wav',
 '/home/DATA/amit_kesari/AMI-diarization-setup/pyannote/amicorpus/ES2002d/audio/ES2002d.Mix-Headset.wav',
 '/home/DATA/amit_kesari/AMI-diarization-setup/pyannote/amicorpus/ES2003a/audio/ES2003a.Mix-Headset.wav',
 '/home/DATA/amit_kesari/AMI-diarization-setup/pyannote/amicorpus/ES2003b/audio/ES2003b.Mix-Headset.wav',
 '/home/DATA/amit_kesari/AMI-diarization-setup/pyannote/amicorpus/ES2003c/audio/ES2003c.Mix-Headset.wav',
 '/home/DATA/amit_kesari/AMI-diarization-setup/pyannote/amicorpus/ES2003d/audio/ES2003d.Mix-Headset.wav',
 '/home/DATA/amit_kesari/AMI-diarization-setup/pyannote/amicorpus/ES2004a/audio/ES2004a.Mix-Headset.wav',
 '/home/DATA/amit_kesari/AMI-diarization-setup

In [5]:
def create_input_manifest(
    audio_filepath,
    rttm_filepath,
    offset=0,
    duration=None,
    label='infer',
    text='-',
    num_speakers=None,
    uem_filepath=None,
    manifest_filepath='/data/input_manifest.json' 
):
    # Create a manifest for input with below format. 
    # {'audio_filepath': /path/to/audio_file, 'offset': 0, 'duration':None, 'label': 'infer', 'text': '-', 
    # 'num_speakers': None, 'rttm_filepath': /path/to/rttm/file, 'uem_filepath'='/path/to/uem/filepath'}
    # if you would like to evaluate with known number of speakers pass the value else None. 
    # If you would like to score the system with known rttms then that should be passed as well, else None
    
    import json
    meta = {
        'audio_filepath': audio_filepath, 
        'offset': 0, 
        'duration':None, 
        'label': 'infer', 
        'text': '-', 
        'num_speakers': num_speakers, 
        'rttm_filepath': rttm_filepath, 
        'uem_filepath' : None
    }
    with open(manifest_filepath,'w') as fp:
        json.dump(meta,fp)
        fp.write('\n')

#     dataframe = pd.DataFrame.from_dict(meta, orient='index')
#     print("Manifest: ", meta)

    
def convert_ami_base():
    # Eg: TS3003a.rttm -> TS3003a.Mix-Headset.rttm
    groundtruth_rttm_list = glob.glob(groundtruth_rttm_filepath + '/test/*.rttm', recursive=True)
    for test_path in groundtruth_rttm_list:
        new_test_path = test_path[:-5]+".Mix-Headset.rttm"
        !mv {test_path} {new_test_path}
        print("new ", new_test_path)

        
        
## RUN ONLY ONCE! Convert rttm to same AMI base name
# convert_ami_base()

In [6]:
# single inialisations
scripts_dir = os.path.join(ROOT, 'scripts/speaker_tasks')
manifest_filepath = os.path.join(data_dir, 'input_manifest.json')
output_dir = os.path.join(data_dir, 'oracle_vad')
os.makedirs(output_dir,exist_ok=True)


MODEL_CONFIG = os.path.join(data_dir,'offline_diarization.yaml')
if not os.path.exists(MODEL_CONFIG):
    config_url = "https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/offline_diarization.yaml"
    MODEL_CONFIG = wget.download(config_url, data_dir)

config = OmegaConf.load(MODEL_CONFIG)
print(OmegaConf.to_yaml(config))

metric = DiarizationErrorRate()
der2 = 0
ami_labels = []
for single_rttm in groundtruth_rttm_list:
    ami_label = single_rttm.split(sep="/")[-1]
    ami_label = ami_label[:-5] # remove .rttm from last
    ami_labels.append(ami_label)
    labels = rttm_to_labels(single_rttm)
    groundtruth = labels_to_pyannote_object(labels)
    num_speakers = Counter([l.split()[-1] for l in labels]).keys().__len__()

    # find the corresponding audio file according to rttm
    single_audio = [audio for audio in audio_list if ami_label in audio][0]
    
    ## -----Visualise data-----
    print("ami_label: ", ami_label)    
    # !cat {single_rttm} # view the sample groundtruth rttm file
    # print("labels: ",labels, groundtruth)
    # print("num_speakers: ", num_speakers)   
    # print("single_audio: ", single_audio)

    # -----Create input manifest and configs-----
    create_input_manifest(audio_filepath=single_audio, rttm_filepath=single_rttm, num_speakers=num_speakers, manifest_filepath=manifest_filepath)
    # change configurations:
    pretrained_speaker_model='titanet_large'
    config.diarizer.manifest_filepath = manifest_filepath
    config.diarizer.out_dir = output_dir # Directory to store intermediate files and prediction outputs
    config.diarizer.speaker_embeddings.model_path = pretrained_speaker_model
    config.diarizer.speaker_embeddings.parameters.window_length_in_sec = 1.5
    config.diarizer.speaker_embeddings.parameters.shift_length_in_sec = 0.75
    config.diarizer.speaker_embeddings.parameters.save_embeddings = True
    config.diarizer.ignore_overlap = False
    config.diarizer.oracle_vad = True # ----> ORACLE VAD 
    # no speaker number init so false
    config.diarizer.clustering.parameters.oracle_num_speakers = False
    # print(OmegaConf.to_yaml(config))
    
    # -----Load model and diarize-----
    oracle_model = ClusteringDiarizer(cfg=config)  
    oracle_model.diarize()
    
    # -----Accumulate DER-----
    pred_labels = rttm_to_labels(output_dir+'/pred_rttms/'+ami_label+'.rttm')
    hypothesis = labels_to_pyannote_object(pred_labels)
    der = metric(groundtruth, hypothesis)
    print(f"Diarization error rate (DER) = {100 * der:.1f}%\n\n")


    
    
"""
Cumulative Results for collar 0.25 sec and ignore_overlap False: 
     FA: 0.0000	 MISS 0.0518	                 Diarization ER: 0.0711	, Confusion ER:0.0193
[NeMo I 2022-07-20 12:00:53 clustering_diarizer:444] Outputs are saved in /home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/oracle_vad directory
Diarization error rate (DER) = 13.8%
"""

name: ClusterDiarizer
num_workers: 4
sample_rate: 16000
batch_size: 64
diarizer:
  manifest_filepath: ???
  out_dir: ???
  oracle_vad: false
  collar: 0.25
  ignore_overlap: true
  vad:
    model_path: null
    external_vad_manifest: null
    parameters:
      window_length_in_sec: 0.15
      shift_length_in_sec: 0.01
      smoothing: median
      overlap: 0.875
      onset: 0.4
      offset: 0.7
      pad_onset: 0.05
      pad_offset: -0.1
      min_duration_on: 0.2
      min_duration_off: 0.2
      filter_speech_first: true
  speaker_embeddings:
    model_path: ???
    parameters:
      window_length_in_sec: 1.5
      shift_length_in_sec: 0.75
      multiscale_weights: null
      save_embeddings: false
  clustering:
    parameters:
      oracle_num_speakers: false
      max_num_speakers: 20
      enhanced_count_thres: 80
      max_rp_threshold: 0.25
      sparse_search_volume: 30
      maj_vote_spk_count: false

ami_label:  TS3003c.Mix-Headset
[NeMo I 2022-07-20 11:58:56 clustering_d

[NeMo W 2022-07-20 11:58:57 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2022-07-20 11:58:57 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2022-07-20 11:58:57 features:200] PADDING: 16
[NeMo I 2022-07-20 11:58:58 label_models:100] loss is Angular Softmax


[NeMo W 2022-07-20 11:58:58 nemo_logging:349] /root/anaconda3/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                    not been set for this class (TopKClassificationAccuracy). The property determines if `update` by
                    default needs access to the full metric state. If this is not the case, significant speedups can be
                    achieved and we recommend setting this to `False`.
                    We provide an checking function
                    `from torchmetrics.utilities import check_forward_no_full_state`
                    that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                    default for now) or if `full_state_update=False` can be used safely.
                    
      warnings.warn(*args, **kwargs)
    


[NeMo I 2022-07-20 11:59:01 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.10.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2022-07-20 11:59:01 speaker_utils:82] Number of files to diarize: 1
[NeMo I 2022-07-20 11:59:01 clustering_diarizer:271] Subsegmentation for embedding extraction: scale0, /home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/oracle_vad/speaker_outputs/subsegments_scale0.json
[NeMo I 2022-07-20 11:59:01 clustering_diarizer:325] Extracting embeddings for Diarization
[NeMo I 2022-07-20 11:59:01 collections:289] Filtered duration for loading collection is 0.000000.
[NeMo I 2022-07-20 11:59:01 collections:293] # 2326 files loaded accounting to # 1 labels


...

[NeMo I 2022-07-20 12:00:46 features:200] PADDING: 16
[NeMo I 2022-07-20 12:00:47 label_models:100] loss is Angular Softmax
[NeMo I 2022-07-20 12:00:47 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.10.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2022-07-20 12:00:47 speaker_utils:82] Number of files to diarize: 1
[NeMo I 2022-07-20 12:00:47 clustering_diarizer:271] Subsegmentation for embedding extraction: scale0, /home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/oracle_vad/speaker_outputs/subsegments_scale0.json
[NeMo I 2022-07-20 12:00:47 clustering_diarizer:325] Extracting embeddings for Diarization
[NeMo I 2022-07-20 12:00:47 collections:289] Filtered duration for loading collection is 0.000000.
[NeMo I 2022-07-20 12:00:47 collections:293] # 2357 files loaded accounting to # 1 labels


100%|██████████| 37/37 [00:02<00:00, 18.14it/s]


[NeMo I 2022-07-20 12:00:49 clustering_diarizer:368] Saved embedding files to /home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/oracle_vad/speaker_outputs/embeddings


100%|██████████| 1/1 [00:01<00:00,  1.98s/it]


[NeMo I 2022-07-20 12:00:53 speaker_utils:460] Cumulative Results for collar 0.25 sec and ignore_overlap False: 
     FA: 0.0000	 MISS 0.0518	                 Diarization ER: 0.0711	, Confusion ER:0.0193
[NeMo I 2022-07-20 12:00:53 clustering_diarizer:444] Outputs are saved in /home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/oracle_vad directory
Diarization error rate (DER) = 13.8%




In [9]:
print(OmegaConf.to_yaml(config))
metric.__dict__

name: ClusterDiarizer
num_workers: 4
sample_rate: 16000
batch_size: 64
diarizer:
  manifest_filepath: /home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/input_manifest.json
  out_dir: /home/DATA/amit_kesari/SD1/NeMo-Nvidia/data/ami_mix_headset/oracle_vad
  oracle_vad: true
  collar: 0.25
  ignore_overlap: false
  vad:
    model_path: null
    external_vad_manifest: null
    parameters:
      window_length_in_sec: 0.15
      shift_length_in_sec: 0.01
      smoothing: median
      overlap: 0.875
      onset: 0.4
      offset: 0.7
      pad_onset: 0.05
      pad_offset: -0.1
      min_duration_on: 0.2
      min_duration_off: 0.2
      filter_speech_first: true
  speaker_embeddings:
    model_path: titanet_large
    parameters:
      window_length_in_sec: 1.5
      shift_length_in_sec: 0.75
      multiscale_weights: null
      save_embeddings: true
  clustering:
    parameters:
      oracle_num_speakers: false
      max_num_speakers: 20
      enhanced_count_thres: 80
      max_rp_

{'metric_name_': 'diarization error rate',
 'components_': {'confusion',
  'correct',
  'false alarm',
  'missed detection',
  'total'},
 'accumulated_': {'correct': 25461.859999999975,
  'total': 30713.92399999997,
  'missed detection': 4469.034000000001,
  'false alarm': 0.0,
  'confusion': 783.0299999999952},
 'results_': [('',
   {'correct': 1767.679999999998,
    'total': 1894.2499999999977,
    'missed detection': 92.48000000000054,
    'false alarm': 0.0,
    'confusion': 34.089999999999336,
    'diarization error rate': 0.066818001847697}),
  ('',
   {'correct': 1796.6149999999964,
    'total': 2530.2599999999948,
    'missed detection': 635.3600000000006,
    'false alarm': 0.0,
    'confusion': 98.28499999999948,
    'diarization error rate': 0.2899484637942352}),
  ('',
   {'correct': 1408.0149999999978,
    'total': 1943.439999999997,
    'missed detection': 466.609999999999,
    'false alarm': 0.0,
    'confusion': 68.81500000000008,
    'diarization error rate': 0.2755037

In [10]:
# display in table
arr = np.asarray(metric.results_)[:,1]
# print("dict: ",arr)

df = pd.DataFrame.from_records(arr)
if 'item' not in df.columns:
    df['item'] = ami_labels
    # add mean summary
    df.loc['mean'] = df.mean()

# Getting the total absolute DER on test set
der2 = df['diarization error rate'].mean()
print(f"Total DER = {100 * der2:.1f}%")    

# saving the dataframe
df.to_csv(output_dir+'/ami_mix_headset_der.csv')
df

[NeMo W 2022-07-20 12:45:03 nemo_logging:349] /tmp/ipykernel_24241/1797775716.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
      df.loc['mean'] = df.mean()
    


Total DER = 15.9%


,correct,total,missed detection,false alarm,confusion,diarization error rate,item
0,1767.68000,1894.25000,92.480000,0.0,34.090000,0.066818,TS3003c.Mix-Headset
1,1796.61500,2530.26000,635.360000,0.0,98.285000,0.289948,EN2002a.Mix-Headset
2,1408.01500,1943.44000,466.610000,0.0,68.815000,0.275504,EN2002b.Mix-Headset
3,2524.41500,3343.64000,738.710000,0.0,80.515000,0.245010,EN2002c.Mix-Headset
4,1830.72000,2675.89000,747.270000,0.0,97.900000,0.315846,EN2002d.Mix-Headset
5,755.27500,923.43000,136.090000,0.0,32.065000,0.182098,ES2004a.Mix-Headset
6,1981.04000,2233.05000,218.840000,0.0,33.170000,0.112855,ES2004b.Mix-Headset
7,1976.62000,2244.47000,231.750000,0.0,36.100000,0.119338,ES2004c.Mix-Headset
8,1664.05000,2006.77000,287.830000,0.0,54.890000,0.170782,ES2004d.Mix-Headset
9,585.34500,695.90000,90.980000,0.0,19.575000,0.158866,IS1009a.Mix-Headset


In [ ]:
groundtruth # to see the labels of each speaker and time intervals, this will show last one only

In [ ]:
# print generated rttm of last test file
!cat {output_dir}/pred_rttms/TS3003d.Mix-Headset.rttm

pred_labels = rttm_to_labels(output_dir+'/pred_rttms/TS3003d.Mix-Headset.rttm')
hypothesis = labels_to_pyannote_object(pred_labels)
hypothesis

In [ ]:
# use spyder

hypothesis_rttm_list = []
hypothesis_rttm_filepath= os.path.join(output_dir, 'pred_rttms')

if test_type == 'test':
    hypothesis_rttm_list = glob.glob(hypothesis_rttm_filepath + '/*.rttm', recursive=True)
    
    for ground_rttm, hyp_rttm in zip(groundtruth_rttm_list, hypothesis_rttm_list):
        print("g h", ground_rttm)
        !spyder {ground_rttm} {hyp_rttm} --per-file

# der2 = abs(metric)

der = metric(groundtruth, hypothesis)
print(f'diarization error rate = {100 * der:.1f}%')

In [11]:
embeddings['TS3003d.Mix-Headset'].shape

{'TS3003d.Mix-Headset': tensor([[ 8.5144e-03, -1.2337e-02,  7.2975e-03,  ..., -1.0834e-02,
          7.5579e-05,  1.6159e-02],
        [ 3.3295e-02,  2.0218e-02, -8.6594e-03,  ...,  2.5040e-02,
         -1.4420e-02, -9.0981e-04],
        [ 5.7831e-03,  6.1874e-03,  2.1305e-03,  ...,  5.9776e-03,
         -3.3386e-02, -8.7814e-03],
        ...,
        [ 3.2135e-02,  2.0432e-02, -1.9196e-02,  ..., -1.2665e-02,
         -7.0129e-02, -4.0588e-02],
        [ 2.1000e-03,  1.0233e-03, -2.3682e-02,  ..., -1.4374e-02,
         -2.0844e-02,  2.0580e-03],
        [ 7.1068e-03,  4.6356e-02, -3.6545e-03,  ...,  2.2240e-03,
          9.1324e-03, -1.3229e-02]])}


torch.Size([2357, 192])

In [ ]:
[NeMo W 2022-07-20 12:00:46 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2022-07-20 12:00:46 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/dev.json
    sample_rate: 16000
    labels: null
    batch_size: 128
    shuffle: false
    time_length: 3
    num_workers: 15
    pin_memory: true

Let's load ground truth RTTM labels and view the reference Annotation timestamps visually

# Storing and Restoring models

Now we can save the whole config and model parameters in a single .nemo and restore from it anytime.

In [ ]:
oracle_model.save_to(os.path.join(output_dir,'diarize.nemo'))

Restore from saved model

In [ ]:
del oracle_model
import nemo.collections.asr as nemo_asr
restored_model = nemo_asr.models.ClusteringDiarizer.restore_from(os.path.join(output_dir,'diarize.nemo'))

# Training 